# Introduction to SQL, part 1

First, let's create a SQLite database with two tables (for students and classes) and then view those records.

* open a terminal
* execute `sqlite3 first.db` to run SQLite and create our database

Notice that `.help` will list usage hints.

We will use two of these to make our table views nice:
* `.header on`
* `.mode column`

Tables in the database can be listed with `.tables`

## Create Students table

Let's create the table for Students:

```
studentId,name,classId
1,Dora,1
2,Daniel,2
3,Mamdooh,1
4,Lana,2
5,Ben,4
```

Execute:
`CREATE TABLE Students (studentId INTEGER, name TEXT, classId INTEGER);`

* We can check it exists with `.tables`.
* Check it's content with `SELECT * FROM Students;`

We need to add content:
* `INSERT INTO Students (studentId, name, classId) VALUES (1,'Dora',1);`

Just to see what's happened, view the content again.

Then try quitting with `.quit` and execute `ls` to see the files in your directory.

Entering lots of rows can get tedious.  Fortunately there are other ways to insert data into tables.
* open up the file `insert-students.sql` to see what's inside
* execute the commands in the SQL script file with `sqlite3 first.db < insert-students.sql`

Start up SQLite again and view the contents of the Students table.

## Create Classes table

For Classes, we will try a third method for inserting data: importing data from a csv file.

* create the table with `CREATE TABLE Classes (classId INTEGER, title TEXT);`
* set the csv mode with `.mode csv`
* import the file with `.import insert-classes.csv Classes`

A few things happened that are non-ideal:

* We imported the CSV header row as a record along with the rest of the data
* SQLite doesn't strictly enforce the type, so we input 'classId' in an integer column
* If we run the import again, there's nothing to stop us from adding duplicate data -- we should really constrain the classId at least to be unique

Delete this table and start over by executing `DROP TABLE Classes;`

Now re-create the table with:
* `CREATE TABLE Classes (classId INTEGER NOT NULL PRIMARY KEY, title TEXT);`
* `.import insert-classes.csv Classes`

Now the import does not match.  Specifically, we already have the column names, so we don't need the header line in the csv file.

* delete the top line in the csv file
* re-import with `.import insert-classes.csv Classes`

Ha, it looks like part of the import was actually already successful and it refuses to add the duplicates.

Check the tables and table contents now that are in the database.  We should be done.

## Survey database

We will come back to students and classes later.  For now, we will start working with a new database.  [This database and materials are adapted from the Databases and SQL Lesson by Software Carpentry, [see here](http://swcarpentry.github.io/sql-novice-survey/).]

You can interact with multiple databases in the same session.
* check your connected databases with `.databases`
* connect to a new database with `ATTACH DATABASE 'survey.db' AS s;`
* check the connected databases again, as well as tables

Files for the new tables are included in the materials as `survey.csv`, `visited.csv`, `person.csv`, and `site.csv`

View the contents at the sqlite console.

## Querying the data

Show all records in a table (e.g. in person table)

`SELECT *
FROM person;`

You can specify columns:

`SELECT family, personal
FROM person;`

Case does not matter:

`seLEcT FamiLY, PERSONal
from perSON;`

Duplication is also permitted

`SELECT personal, id, family, family, id, personal
FROM person;`

-- -- EXERCISE -- 
Write a query that selects only the 'name' from the 'Site' table
-- -- --

### Dealing with duplicates and sorting...

Look at the quant column of Survey:

`SELECT quant
FROM survey;`

Grab only the unique values

`SELECT DISTINCT quant
FROM survey;`

Grab unique pairs

`SELECT DISTINCT quant, taken
FROM survey;`

Unique pairs can also be ordered

`SELECT DISTINCT quant, taken
FROM survey
ORDER BY quant;`

And the order can be switched from ascending (ASC) to descending (DESC)

`SELECT DISTINCT quant, taken
FROM survey
ORDER BY quant, taken DESC;`

Text can also be ordered

`SELECT *
FROM person
ORDER BY id;`

`SELECT *
FROM person
ORDER BY id DESC;`

There are some "NULL" values that are also in here....

`SELECT DISTINCT quant, person
FROM survey
ORDER BY quant ASC;`

`SELECT DISTINCT quant, person
FROM survey
ORDER BY quant ASC, person;`

-- -- EXERCISE -- 
Write a query that selects the distinct dates shown in the 'Visited' table
-- -- --

-- -- EXERCISE -- 
Write a query that shows the full names of scientists in the 'Person' table, ordered by family name
-- -- --

### Filtering

Select records that match a certain criteria with WHERE

`SELECT *
FROM visited
WHERE site = 'DR-1';`

The column name doesn't have to match the column used for selection

`SELECT id
FROM visited
WHERE site = 'DR-1';`

AND and OR can be used as conditionals

`SELECT *
FROM visited
WHERE (site = 'DR-1') AND (dated < '1930-01-01');`

`SELECT *
FROM survey
WHERE (person = 'lake') OR (person = 'roe');`

Conditional relationships can also use:
* IN to test group membership
* BETWEEN to test a range
* LIKE with wildcards for contained substrings ('%' means anything and '_' means single character)

`SELECT *
FROM survey
WHERE person IN ('lake','roe');`

Watch out for ordering of evaluation

`SELECT *
FROM survey
WHERE quant = 'sal' AND person = 'lake' OR person = 'roe';`

`SELECT *
FROM survey
WHERE quant = 'sal' AND (person = 'lake' OR person = 'roe');`

Wildcard example

`SELECT *
FROM visited
WHERE site LIKE 'DR%';`

DISTINCT can be applied to paired results after the condition has been met

`SELECT DISTINCT person, quant
FROM survey
WHERE person = 'lake' OR person = 'roe';`

-- -- EXERCISE -- 
Say we want all sites that lie within 48 degrees of the equator, that is, latitutide from -48 to 48
Fix this:

`SELECT *
FROM site
WHERE (lat > -48) OR (lat < 48);`
-- -- --

-- -- EXERCISE -- 
Normalized salinity reading should be between 0.0 and 1.0.
Select records from the 'Survey' table that are outside this range:
-- -- --

### Calculating new values

Perhaps 'roe' was misreporting salinity values

`SELECT *
FROM survey
WHERE quant = 'sal'
ORDER BY person DESC;`

We can divide by 100 but ....

`SELECT person, quant, reading/100.0
FROM survey
WHERE quant = 'sal'
ORDER BY person DESC;`

Simple math operations can be applied to columns

`SELECT 1.05*reading
FROM survey
WHERE quant = 'rad';`

And it can be useful to temporarily rename the output column

`SELECT 1.05*reading AS 'Radiation corrected by 5%'
FROM survey
WHERE quant = 'rad';`

The reported temperature is in Fahrenheit

`SELECT reading
FROM survey
WHERE quant = 'temp';`

A useful calculation might be to return temperature in Celsius

`SELECT round(5 * (reading - 32) / 9, 2) as 'Temp (C)'
FROM survey
WHERE quant = 'temp';`

String concatenation is via "||"

`SELECT personal || ' ' || family
FROM person;`

Back to salinity:
-- We can divide by 100 for 'roe' ....

`SELECT person, quant, reading/100.0
FROM survey
WHERE quant = 'sal' AND person = 'roe';`

Combine this now with the notion of unions

`SELECT *
FROM person
WHERE id = 'dyer'
UNION
SELECT *
FROM person
WHERE id = 'roe';`

Compare UNION vs UNION ALL

`SELECT *
FROM person
WHERE id = 'dyer'
UNION ALL
SELECT *
FROM person;`

-- -- EXERCISE -- 
Use UNION to show the salinity/100 for 'roe' and original salinity readings for everyone else
-- -- --

### A few details about NULL

`SELECT *
FROM visited;`

NULL is a special value, and not equal to, for example, a missing or 'None' value

`SELECT *
FROM visited
WHERE dated = 'None';`

Equality must be checked with IS NULL or IS NOT NULL

`SELECT *
FROM visited
WHERE dated IS NULL;`

`SELECT *
FROM visited
WHERE dated IS NOT NULL;`

Logical operators will not return anything for NULL values

`SELECT *
FROM survey
WHERE quant = 'sal' AND person != 'lake';`

NULL must be explicitly checked for

`SELECT *
FROM survey
WHERE quant = 'sal' AND (person != 'lake' OR person IS NULL);`

-- -- EXERCISE -- 
Write a query that sorts records in 'Visited' by date and omits unknown dates
-- -- --

-- -- EXERCISE --
What do you think this query will produce?

`SELECT *
FROM visited
WHERE dated IN ('1927-02-08', NULL);`

-- What is a fix? --
-- -- --

### AGGREGATE CALCULATIONS, GROUP BY, and HAVING

You can use basic arithmetic operations to get summary data for the whole table.  These include COUNT, AVG, SUM, MEAN, MIN, MAX.

`SELECT COUNT(*) 
FROM survey;`

`SELECT MIN(*) 
FROM survey
WHERE quant = 'temp';`

Summaries for groups within the table can be collected via GROUP BY

`SELECT person, COUNT(*) 
FROM survey 
GROUP BY person;`

... and if you don't want NULL values included:

`SELECT person, COUNT(*) 
FROM survey 
WHERE person IS NOT NULL
GROUP BY person;`

Do note that WHERE must precede GROUP BY.  The WHERE clause filters individual records, and the following will give an error:

`SELECT person, COUNT(*) 
FROM survey 
GROUP BY person
WHERE person IS NOT NULL;`

Filtering for groups can be achieved via the HAVING clause:

`SELECT taken,count(*)
FROM survey
GROUP BY taken
HAVING count(*) > 2;`

Sometimes this can start to stretch the mind, so let's check out the slightly simpler query:

`SELECT taken,count(*)
FROM survey
GROUP BY taken;`

How about:

`SELECT *,count(*)
FROM survey
GROUP BY taken
HAVING count(*) > 2;`

The result from `GROUP BY taken HAVING count(*) > 2` doesn't have multiple rows, so SQL here simply returns values for other columns that are the first entries of rows matching the appropriate 'taken' column.

To get all the entries for given values of 'taken' that have a count greater than 2, we can use a sub-query, i.e., put a select statement in parentheses and use it as if it was another table.

`SELECT * 
FROM survey 
WHERE taken IN (
    SELECT taken 
    FROM survey 
    GROUP BY taken 
    HAVING count(*) > 2
);`

If there were lots of values for 'taken', this might be a slow query.  You can speed it up by using a JOIN to similar effect:

`SELECT * 
FROM survey 
INNER JOIN
    (SELECT taken 
    FROM survey 
    GROUP BY taken 
    HAVING count(*) > 2) 
AS t 
USING (taken);`

So.... what are JOINs???